In [11]:
import numpy as np, plotly.graph_objects as go
import ipywidgets as w

# --- Two-tap channel: h(t) = a0 δ(t) + a1 δ(t-τ) ---
def H_mag_two_tap(f, a0, a1, tau):
    j = 1j
    H = a0 + a1*np.exp(-j*2*np.pi*f*tau)
    return np.abs(H)

def null_frequencies(tau, fmax):
    """Return all f_null=(1+2k)/(2*tau) within [0, fmax]."""
    if tau <= 0:
        return np.array([])
    df = 1.0/tau
    # k >= 0: f = (1+2k)/(2*tau) = df*(k + 0.5)
    kmax = int(np.floor((fmax/df) - 0.5))
    if kmax < 0:
        return np.array([])
    k = np.arange(kmax+1)
    return df*(k + 0.5)

# Frequency axis (editable via slider)
fmax = 1e6  # 1 MHz
N = 20001
f = np.linspace(0, fmax, N)

# --- Controls ---
a0  = w.FloatSlider(value=1.0, min=0, max=2, step=0.01, description='a0', readout_format='.2f')
a1  = w.FloatSlider(value=1, min=0, max=2, step=0.01, description='a1', readout_format='.2f')
tau = w.FloatSlider(value=5.0, min=0, max=50, step=0.01, description='τ [μs]', readout_format='.2f')

fmax_slider = w.FloatSlider(value=fmax*1e-6, min=0.05, max=5.0, step=0.05,
                            description='f max [MHz]', readout_format='.2f')
scale = w.ToggleButtons(options=[('Linear','lin'), ('Log (dB)','log')],
                        value='lin', description='Scale:')
show_nulls = w.ToggleButtons(options=[('Nulls ON', True), ('Nulls OFF', False)],
                             value=True, description='Markers:')

# --- Figure (trace 0 = |H(f)|; null markers are shapes, not traces) ---
fig = go.FigureWidget(
    data=[go.Scatter(x=f*1e-6, y=np.zeros_like(f), name='|H(f)|')],
    layout=go.Layout(
        xaxis=dict(title='Frequency (MHz)'),
        yaxis=dict(title='|H(f)|'),
        height=520, margin=dict(l=60, r=10, t=40, b=50),
        shapes=[], annotations=[]
    )
)

def update(*_):
    # Frequency grid if span changed
    fmax = fmax_slider.value*1e6
    f_new = np.linspace(0, fmax, N)

    # Compute |H(f)|
    Hm = H_mag_two_tap(f_new, a0.value, a1.value, tau.value*1e-6)

    # Y data + axis label
    if scale.value == 'log':
        y = 20*np.log10(np.maximum(Hm, 1e-12))
        ylab = '|H(f)| [dB]'
    else:
        y = Hm
        ylab = '|H(f)|'

    # Build shapes for nulls
    tau_s = tau.value*1e-6
    fn = null_frequencies(tau_s, fmax) if show_nulls.value else np.array([])
    # vertical lines as shapes
    shapes = []
    for ff in fn:
        x = ff*1e-6
        shapes.append(dict(
            type='line', xref='x', yref='paper',
            x0=x, x1=x, y0=0, y1=1,
            line=dict(width=1, dash='dot')
        ))

    # Annotation with Δf and first few nulls
    if tau_s > 0:
        df = 1.0/tau_s
        txt = f"Δf = 1/τ = {df*1e-6:.3f} MHz"
        if fn.size:
            first = fn[:3]*1e-6
            txt += " | f_null ≈ " + ", ".join(f"{v:.3f}" for v in first) + " MHz"
    else:
        txt = "Δf undefined for τ=0"

    with fig.batch_update():
        fig.data[0].x = f_new*1e-6
        fig.data[0].y = y
        fig.update_yaxes(title=ylab)
        fig.update_xaxes(range=[0, fmax*1e-6])
        fig.layout.shapes = tuple(shapes)
        fig.layout.annotations = (dict(
            x=0.01, y=1.08, xref='paper', yref='paper',
            text=txt, showarrow=False, font=dict(size=12)
        ),)

for ctl in (a0, a1, tau, fmax_slider, scale, show_nulls):
    ctl.observe(update, 'value')

update()
w.VBox([
    w.HBox([a0, a1, tau]),
    w.HBox([scale, show_nulls, fmax_slider]),
    fig
])
